# USCIS Case Status Info for multiple cases

### Imports

In [1]:
from requests import post
from bs4 import BeautifulSoup
from IPython.core.display import display
from ipywidgets import widgets
import pandas as pd
import numpy as np
import re
from datetime import datetime

### Enter USCIS receipt number(s)

In [2]:
receipt_inputs = widgets.Textarea(value = 'MSC1890265705\nMSC1890265706\nMSC1890265762\nMSC1890265763\nMSC1791508027',
                                  placeholder = 'Enter USCIS Receipt Numbers, one on each line',
                                  description = 'Receipt #s',
                                  disabled = False)
display(receipt_inputs)

Textarea(value='MSC1890265705\nMSC1890265706\nMSC1890265762\nMSC1890265763\nMSC1791508027', description='Recei…

### Get case status information for a given receipt

In [3]:
def get_case_status(rcpt, http_status = False):
    # USCIS case status form
    endpoint = "https://egov.uscis.gov/casestatus/mycasestatus.do"
    
    # Get response from POST request
    response = post(endpoint,
                    data={'appReceiptNum': rcpt, 
                          'caseStatusSearchBtn': 'CHECK+STATUS'})
    
    # Print response status code, if specified
    if http_status:
        print("Response:", response.status_code, response.reason)
    
    # Response is HTML. Use BS4 to parse it and get relevant info
    soup = BeautifulSoup(response.text, 'html.parser')
    relevant_info = soup.find("div", {"class": "rows text-center"})
    case_summary = str(relevant_info.h1.get_text())
    case_details = str(relevant_info.p.get_text())
    
    # Get application type
    form_type_search = re.search("Form ([^,]+)", case_details)
    if form_type_search:
        form_type = form_type_search.group(1)
    else:
        form_type = None
    
    # Get last update date
    last_update_search = re.search("^On (\w+\s\d{1,2},\s+\d{4})", case_details)
    if last_update_search:
        last_update = last_update_search.group(1)
        # Calculate days since last update
        days_since_update = (datetime.now() - datetime.strptime(last_update, '%B %d, %Y')).days
    else:
        last_update = None
        days_since_update = None
    
    # Return list of case summary and details
    return[rcpt, case_summary, form_type, last_update, days_since_update, case_details]

### Insert case status information into a Pandas DataFrame


In [4]:
rcpts = receipt_inputs.value.split('\n')
case_statuses = pd.DataFrame(index = np.arange(0, len(rcpts)), 
                             columns=['RECEIPT #', 'STATUS', 'FORM TYPE', 'LAST UPDATE', 'DAYS SINCE LAST UPDATE', 'DETAILS'])
for r in np.arange(0, len(rcpts)):
    case_statuses.loc[r] = get_case_status(rcpts[r]) # Append results to DF

### Results

In [5]:
pd.set_option('display.max_colwidth', -1) # Don't truncate long values
case_statuses

,RECEIPT #,STATUS,FORM TYPE,LAST UPDATE,DAYS SINCE LAST UPDATE,DETAILS
0,MSC1890265705,Response To USCIS' Request For Evidence Was Received,I-485,"January 26, 2018",108,"On January 26, 2018, we received your response to our Request for Evidence for your Form I-485, Application to Register Permanent Residence or Adjust Status, Receipt Number MSC1890265705. Our National Benefits Center office will begin working on your case again. We will mail you a decision or notify you if we need something from you. If you move, go to www.uscis.gov/addresschange to give us your new mailing address."
1,MSC1890265706,Case Was Received,I-130,"November 20, 2017",175,"On November 20, 2017, we received your Form I-130, Petition for Alien Relative, Receipt Number MSC1890265706, and sent you the receipt notice that describes how we will process your case. Please follow the instructions in the notice. If you do not receive your receipt notice by December 20, 2017, please call the USCIS Contact Center at 1-800-375-5283. If you move, go to www.uscis.gov/addresschange to give us your new mailing address."
2,MSC1890265762,Case Was Received,I-765,"November 21, 2017",174,"On November 21, 2017, we received your Form I-765, Application for Employment Authorization, Receipt Number MSC1890265762, and sent you the receipt notice that describes how we will process your case. Please follow the instructions in the notice. If you do not receive your receipt notice by December 21, 2017, please call the USCIS Contact Center at 1-800-375-5283. If you move, go to www.uscis.gov/addresschange to give us your new mailing address."
3,MSC1890265763,Case Was Received,I-131,"November 21, 2017",174,"On November 21, 2017, we received your Form I-131, Application for Travel Document, Receipt Number MSC1890265763, and sent you the receipt notice that describes how we will process your case. Please follow the instructions in the notice. If you do not receive your receipt notice by December 21, 2017, please call the USCIS Contact Center at 1-800-375-5283. If you move, go to www.uscis.gov/addresschange to give us your new mailing address."
4,MSC1791508027,CASE STATUS,None,None,None,At this time USCIS cannot provide you with information for your case. Please contact the USCIS Contact Center at 1-800-375-5283 for additional information.
